# Importar Librerias

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, inspect, text
from sqlalchemy.exc import SQLAlchemyError


### Crear el Engine

In [2]:
engine = create_engine('sqlite:///ejercicio_database.db')

In [4]:
df = pd.read_csv('clientes_banco.csv')
df.head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


### Guardar en tabla sql

In [8]:
# df.to_sql('clientes_financiera', engine, index=False)

In [7]:
inspector = inspect(engine)
print(inspector.get_table_names())

['clientes_financiera']


In [7]:
pd.read_sql_table('clientes_financiera', con=engine.connect()).head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


### Actualizacion del Registro ID 6840104

In [9]:
query = "UPDATE clientes_financiera SET Rendimiento_anual=300000.0 WHERE ID_Cliente=6840104"
try:
    r_set=engine.connect().execute(text(query))
except SQLAlchemyError as e:
    print(e)
else:
    print("#registros actualizados:", r_set.rowcount)

#registros actualizados: 1


### Eliminar Registro ID 5008809

In [8]:
query = "DELETE FROM clientes_financiera WHERE ID_Cliente=5008809"
try:
    r_set=engine.connect().execute(text(query))
except SQLAlchemyError as e:
    print(e)
else:
    print("#Registros borrados:", r_set.rowcount)

#Registros borrados: 1


### Creando Nuevo Registro en la BD

In [4]:
query = 'INSERT INTO clientes_financiera (ID_Cliente, Edad, Grado_estudio, Estado_civil, ' \
        'Tamaño_familia, Categoria_de_renta, Ocupacion, Años_empleado, ' \
        'Rendimiento_anual, Tiene_carro, Vivienda) ' \
        'VALUES (6850985, 33, "Doctorado", "Soltero", 1, "Empleado", "TI", ' \
        '2, 290000, 0, "Casa/Departamento propio")'
        
             
try:
    r_set=engine.connect().execute(text(query))
except SQLAlchemyError as e:
    print(e)
else:
    print("#Registros insertados:", r_set.rowcount)        

#Registros insertados: 1


### Verificar los Cambios Realizados

In [5]:
pd.read_sql_table("clientes_financiera", con=engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio
438459,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438460,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438461,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio
